# PTF_MVT 03: AZEC Processor Testing

**Purpose**: Test complete AZEC processor (Bronze → Silver)

**Tests**: POLIC_CU, movements, suspension, capital enrichment

---

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))
print(f"Project root: {project_root}")

In [ ]:
from pyspark.sql import SparkSession
# from azfr_fsspec_utils import fspath
# import azfr_fsspec_abfs

# azfr_fsspec_abfs.use()

spark = SparkSession.builder \
    .appName("PTF_MVT_AZEC_Processor") \
    .getOrCreate()

print(f"✓ Spark {spark.version}")

## 1. Initialize

In [ ]:
from utils.loaders.config_loader import ConfigLoader
from utils.logger import PipelineLogger

config = ConfigLoader(str(project_root / "config" / "config.yml"))
logger = PipelineLogger("azec_test")

VISION = "202509"
print(f"Vision: {VISION}")

## 2. Run AZEC Processor

In [ ]:
from src.processors.ptf_mvt_processors.azec_processor import AZECProcessor

try:
    azec_processor = AZECProcessor(spark, config, logger)
    df_azec = azec_processor.run(VISION)
    
    if df_azec is not None:
        print(f"✓ AZEC Processor: {df_azec.count():,} rows")
        print(f"  Columns: {len(df_azec.columns)}")
    else:
        print("⚠ AZEC Processor returned None (data may not exist)")
except Exception as e:
    print(f"⚠ AZEC: {e}")
    df_azec = None

## 3. Verify Key Columns

In [ ]:
if df_azec is not None:
    # Movements
    df_azec.select('police', 'nbafn', 'nbres', 'nbptf').show(3)
    
    # Suspension
    if 'nbj_susp_ytd' in df_azec.columns:
        df_azec.select('police', 'nbj_susp_ytd').show(3)

## Summary

In [ ]:
print("="*50)
print("AZEC PROCESSOR TESTING COMPLETE")
print("="*50)
print(f"Result: {'✓ Success' if df_azec is not None else '⚠ Optional - data may not exist'}")
print("→ Next: Notebook 04 - Consolidation")